In [2]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
stop_words = stopwords.words('english')
nltk.download('punkt') # Pre-trained tokenizer to help tokenize sentences

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dask.dataframe as dd #  Initially thought I might need this due to all sorts of errors with large datasets. Manged to fix them
import tensorflow as tf

In [4]:
def processtext(x):
    filtered_list = []
    text_tokens = word_tokenize(str(x))
    for w in text_tokens:
        if w not in stop_words:
            filtered_list.append(w)
    filtered_sentence = (" ").join(filtered_list)
    filtered_sentence = re.sub(r'[^\w]', ' ', filtered_sentence)
    filtered_sentence = re.sub(r"\d", "", filtered_sentence)
    return filtered_sentence

In [5]:
import chardet
with open("C:/Users/andre/Documents/tweets.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}

In [50]:
df = pd.read_csv("C:/Users/andre/Documents/tweets2.csv", encoding = 'UTF-8',
                names = ["polarity", "tweet"])
df.head()

,polarity,tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [51]:
df['tweet'] = df['tweet'].apply(processtext)

In [54]:
df.head()

,polarity,tweet
0,0,switchfoot http twitpic Awww bummer shoulda Da...
1,0,upset update Facebook texting might result Sch...
2,0,Kenichan dived many times ball Managed save re...
3,0,whole body feels itchy like fire
4,0,nationwideclass behaving


In [52]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
        
    return input_txt

In [55]:
df['tweet'] = df['tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [56]:
df.head()

,polarity,tweet
0,0,switchfoot http twitpic Awww bummer shoulda Da...
1,0,upset update Facebook texting might result Sch...
2,0,Kenichan dived many times ball Managed save re...
3,0,whole body feels itchy like fire
4,0,nationwideclass behaving


In [57]:
from sklearn.model_selection import train_test_split
y = df['polarity']
X = df['tweet']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 7)
print('Train data Samples:', X_train.shape[0])
print('Test data Samples:', X_test.shape[0])

Train data Samples: 734003
Test data Samples: 314573


In [58]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
maxlen = 100
tk = Tokenizer(num_words=5000)
vocab_size = len(tk.word_index) + 1

tk.fit_on_texts(X_train)
tk.fit_on_texts(X_test)

# For Sequences
X_train_seq = tk.texts_to_sequences(X_train)
X_test_seq = tk.texts_to_sequences(X_test)

#Padding to make equal length sentences
X_train_padded = pad_sequences(X_train_seq, padding = "post")
X_test_padded = pad_sequences(X_test_seq, padding = "post")

In [59]:
tk.word_index

{'like': 1,
 'work': 2,
 'good': 3,
 'today': 4,
 'going': 5,
 'quot': 6,
 'back': 7,
 'http': 8,
 'really': 9,
 'time': 10,
 'know': 11,
 'still': 12,
 'want': 13,
 'miss': 14,
 'love': 15,
 'home': 16,
 'night': 17,
 'think': 18,
 'well': 19,
 'last': 20,
 'need': 21,
 'much': 22,
 'feel': 23,
 'wish': 24,
 'tomorrow': 25,
 'sorry': 26,
 'sleep': 27,
 'morning': 28,
 'hope': 29,
 'could': 30,
 'twitter': 31,
 'would': 32,
 'right': 33,
 'hate': 34,
 'just': 35,
 'though': 36,
 'thanks': 37,
 'tonight': 38,
 'getting': 39,
 'great': 40,
 'come': 41,
 'make': 42,
 'haha': 43,
 'week': 44,
 'school': 45,
 'sick': 46,
 'happy': 47,
 'better': 48,
 'even': 49,
 'dont': 50,
 'cant': 51,
 'yeah': 52,
 'people': 53,
 'tired': 54,
 'days': 55,
 'never': 56,
 'working': 57,
 'weekend': 58,
 'watching': 59,
 'feeling': 60,
 'next': 61,
 'wait': 62,
 'nice': 63,
 'long': 64,
 'soon': 65,
 'take': 66,
 'that': 67,
 'little': 68,
 'already': 69,
 'find': 70,
 'twitpic': 71,
 'phone': 72,
 'please'

In [72]:
# Simple Neural Network
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, ),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model.summary())

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [71]:
history = model.fit(X_train_padded, y_train, batch_size = 256, epochs=10, verbose=2, validation_data=(X_test_padded, y_test))

Epoch 1/10


InvalidArgumentError:  indices[0,0] = 855 is not in [0, 1)
	 [[node sequential_6/embedding_6/embedding_lookup (defined at <ipython-input-71-123b63ea3ac3>:1) ]] [Op:__inference_train_function_5314]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_6/embedding_6/embedding_lookup:
 sequential_6/embedding_6/embedding_lookup/5046 (defined at C:\ProgramData\Anaconda3\lib\contextlib.py:113)

Function call stack:
train_function


In [63]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 100),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(6, activation = 'softmax')
])
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 100)         100       
_________________________________________________________________
global_average_pooling1d_4 ( (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                3232      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 198       
Total params: 3,530
Trainable params: 3,530
Non-trainable params: 0
_________________________________________________________________


In [64]:
history = model.fit(X_train_padded, y_train, batch_size = 256, epochs=10, verbose=2, validation_data=(X_test_padded, y_test))

Epoch 1/10


InvalidArgumentError:  indices[120,0] = 11 is not in [0, 1)
	 [[node sequential_4/embedding_4/embedding_lookup (defined at <ipython-input-64-123b63ea3ac3>:1) ]] [Op:__inference_train_function_3769]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_4/embedding_4/embedding_lookup:
 sequential_4/embedding_4/embedding_lookup/3524 (defined at C:\ProgramData\Anaconda3\lib\contextlib.py:113)

Function call stack:
train_function
